# Algorithm Analysis

This notebook describes Algorithmic Analysis in Python and Recursion. It covers the following:

1. Algorithm Analysis
2. "Big-Oh" Notation
3. Computing Runtimes
4. Loop Invariants
5. Factorial Function
6. Binary Search
7. Disk Utility Function
8. Inefficient Use of Recursion
9. Different Kinds of Recursion

For each concept, there are Python examples which help illustrate the ideas.

## 1. Algorithm Analysis

![](images/algo_intro.png)

## 2. "Big-Oh" Notation

![](images/orders_table.png)

See [my notes on computational complexity theory](https://drive.google.com/open?id=1OOlB41DjdCGSzuiSFz9DlQr2TlTSp7aE) for some examples of these.

![](images/orders_table2.png)

## 3. Computing Runtimes

We have the following examples:
* Three way set disjointness
* element Uniqueness

Notice that in each version of these algorithms, we count the steps (to empirically verify the different in runtime)

### Three way set disjointness

Consider the following algorithms for computing whether 3 sets do not share a common element. 

In [1]:
def disjoint1(A, B, C):
    """Return True if there is no element common to all three lists."""
    i=0
    for a in A:
        for b in B:
            for c in C:
                i += 1
                if a == b == c:
                    print('took {} steps'.format(i))
                    return False # we found a common value
    print('took {} steps'.format(i))
    return True

In [2]:
def disjoint2(A, B, C):
    """Return True if there is no element common to all three lists."""
    i=0
    for a in A:
        for b in B:
            i += 1
            if a == b: # only check C if we found match from A and B
                for c in C:
                    i += 1
                    if a == c: # (and thus a == b == c)
                        print('took {} steps'.format(i))
                        return False # we found a common value
    print('took {} steps'.format(i))
    return True # if we reach this, sets are disjoint

`disjoint1` should take $O(n^3)$ steps to run, why?

* the outermost loop is $O(n)$
* for each of the $O(n)$ iterations, another loop is run, which is $O(n)$ (so $O(n^2)$ total)
* for each of $O(n^2)$ iterations, a third loop is run, for a total of $O(n^3)$ iterations

We verify this empirically:

In [3]:
disjoint1({1,2,3,4,5},{6,7,8,9,10},{11,12,13,14,15})

took 125 steps


True

`disjoint2` should take $O(n^2)$ steps to run, why?

* the outermost loop is $O(n)$
* for each of the $O(n)$ iterations, another loop is run, which is $O(n)$ (so $O(n^2)$ total)
* Not always will the innermost loop run. It will run at most $n$ times (if $A\subseteq B$ or $B \subseteq A$)
* For each of the (at most) $O(n)$ iterations when $a==b$, a third loop is run, for a total of $O(n^2)$ iterations

Hence, $O(n^2)+O(n^2)$ is $O(n^2)$

We validate this empirically:

1) When all 3 sets disjoint scenerio, this does very well since the inner loop never executes.

In [4]:
disjoint2({1,2,3,4,5},{6,7,8,9,10},{11,12,13,14,15})

took 25 steps


True

2) Worst case scenerio. Notice the inner loop executes all $n$ times

In [5]:
disjoint2({1,2,3,4,5},{1,2,3,4,5},{11,12,13,14,15})

took 50 steps


True

3) Best case scenerio. A common value is immediately found

In [6]:
disjoint2({1,2,3,4,5},{1,2,3,4,5},{1,2,3,4,5})

took 2 steps


False

### Element Uniqueness

Consider the following algorithm for computing whether there are any duplicate elements in a sequence

In [7]:
def unique2(S):
    """Return True if there are no duplicate elements in sequence S."""
    i = 0
    for j in range(len(S)):
        for k in range(j+1, len(S)):
            i += 1
            if S[j] == S[k]:
                print('took {} steps'.format(i))
                return False # found duplicate pair
    print('took {} steps'.format(i))
    return True # if we reach this, elements were unique

`unique2` should take $O(n^2)$ steps to run. Why?
* Notice that during the $j^{th}$ iteration of the outer loop, the inner loop is executed at most $j-1$ times.
* So the total number of iterations of the loops are: $1+2+...+(n-1) = \frac{n(n-1)}{2}-n$, which is $O(n^2)$

We confirm this empirically

In [8]:
unique2('abcdefghij')

took 45 steps


True

Indeed, it took $\frac{10\cdot9}{2}=45$ steps to run

Aside: We can see the low level code using the `dis` module

In [9]:
#import dis
#dis.dis(disjoint1)

## 4. Loop Invariants

We can use loop invariants to prove things about a loop $L$.
To use it, define $L$ in terms of $L_0, L_1, ..., L_k$ where:
* $L_0$ is true before the loop begins
* If $L_{j-1}$ is true before iteration $j$, then $L_j$ will be true after iteration $j$
* $L_k \Rightarrow L$

Note: the loop need not have a predefined length, so $k$ just needs to be the iteration at which the algorithm terminates

Consider the following example.

In [10]:
def find(S, val):
    """Return smallest index j such that S[j] == val, or -1 if no such element."""
    n = len(S)
    j = 0
    while j < n:
        if S[j] == val:
            return j # a match was found at index j
        j += 1
    return -1

Let $L$ be "the following algorithm is correct". We use loop invariants to prove this statement.

Let $L_j$  be "$S[i]\ne$ val $\forall i\le j$ at the beginning of iteration j of the while loop" 

We have that:
* $L_0$  is true trivially, since there are no elements to compare to val
* Let $L_{j-1}$  be true before iteration $j$. There are two cases:
    * If $S[j]\ne$ val, then $L_j$  is satisfied, since $S[i]\ne$ val  $\forall i \le j-1$ and for j
    * If $S[j]=$val, then the algorithm terminates, and $L_j\Rightarrow L$ (since the algorithm correctly returns j)
* If $L_n$  is true,  val $\notin S$, so $L_n \Rightarrow L$ (since the algorithm correctly returns −1)

## 5. Factorial Function

The factorial function is one of the easiest recursions, so we analyze it first.

In [11]:
def factorial(n):
    if n==0:
        return 1
    else:
        return n*factorial(n-1)

Each time a function is called, a structure called a **frame** is created, which contains information about the execution of the function (including a namespace).

When there is nested function calls, the "outer" function call is suspended, and the frame stores where to resume once the "inner" function call finishes.

_The inner call will have its own frame!_



![](images/factorial_trace.png)

In general, for a recursive algorithm, we add up the number of operations for each function activation.

For the Factorial Algorithm, there are n+1 activations:
* since `factorial(n)` is first called;
* which calls `factorial(n-1)`
* which calls `factorial(n-2)`
* ...
* which calls `factorial(0)`

Each is $O(1)$, so the algorithm is $O(n)$

## 6. Binary Search

Suppose we want to find a target value within a sequence of $n$ elements.
* We can loop over elements until the value is found $\rightarrow$ Sequential Search
* But this is $O(n)$ …
* For an sorted and indexable sequence, we can use Binary Search

Consider the following binary search algorithm

In [12]:
def binary_search(data, target, low, high):
    """
    Return True if target is found in indicated portion of a 
    Python list. The search only considers the portion from 
    data[low] to data[high] inclusive
    """
    global i
    i += 1
    if low > high:
        return False
    else:
        mid = (low + high)//2
        print('low : {}, mid: {}, high: {}'.format(low, mid, high))  # track progress
        if target == data[mid]:
            return True
        elif target < data[mid]:
            return binary_search(data, target, low, mid-1)
        elif target > data[mid]:
            return binary_search(data, target, mid+1, high)

`binary_search` should take $O( \log n )$ steps to run. Why?

There are at most $\lfloor\log ⁡n\rfloor+1$ recursive calls for a sequence with n elements, each $O(1) \Rightarrow$ the algorithm is $O( \log⁡ n)$
* Why? First, notice that, after the first call, the number of remaining candidates is $\le \frac{n}{2}$.
* After the $r^{th}$ step, there are $\le \frac{n}{2^r}$ candidates left.
* In the worst case, the number of steps are smallest $r$ s.t. there is at most one candidate left: 
    * $1>\frac{n}{2^r} \Rightarrow r > \log ⁡n \Rightarrow r=\lfloor\log ⁡n\rfloor+1$

See the image for a visualization of the execution of this function

![](images/binary_search.png)

We test the code empirically:

In [13]:
i=0
data = [2,4,5,7,8,9,12,14,17,19,22,25,27,28,33,37]

binary_search(data,-1,0,15)
print('took {} steps'.format(i))

low : 0, mid: 7, high: 15
low : 0, mid: 3, high: 6
low : 0, mid: 1, high: 2
low : 0, mid: 0, high: 0
took 5 steps


In [14]:
import math
math.floor(math.log2(len(data)))+1

5

Indeed, this takes $\lfloor \log ⁡16 \rfloor+1 = 4+1 = 5$

## 7. Disk Utility

Consider the following program to calculate disk usage (like `du` in Unix)

In [15]:
import os

def disk_usage(path):
    """Return the number of bytes used by a file/folder and any descendents."""
    total = os.path.getsize(path)                    # account for direct usage
    if os.path.isdir(path):                          # if this is a directory,
        for filename in os.listdir(path):            # then for each child:
            childpath = os.path.join(path, filename) # compose full path to child
            total += disk_usage(childpath)           # add child’s usage to total
    print ( '{0:<7}'.format(total), path)            # descriptive output (optional)
    return total                                     # return the grand total

In [16]:
disk_usage('courses') # note that this includes hidden directories/files

0       courses/cs016/homeworks/hw1.txt
0       courses/cs016/homeworks/.ipynb_checkpoints/hw1-checkpoint.txt
3       courses/cs016/homeworks/.ipynb_checkpoints
0       courses/cs016/homeworks/hw3.txt
0       courses/cs016/homeworks/hw2.txt
9       courses/cs016/homeworks
0       courses/cs016/programs/pr3.txt
0       courses/cs016/programs/pr2.txt
0       courses/cs016/programs/pr1.txt
0       courses/cs016/programs/.ipynb_checkpoints/pr1-checkpoint.txt
3       courses/cs016/programs/.ipynb_checkpoints
9       courses/cs016/programs
2       courses/cs016/.ipynb_checkpoints
2       courses/cs016/grades
28      courses/cs016
2       courses/.ipynb_checkpoints
2       courses/cs252/grades
0       courses/cs252/projects/papers/sellhigh.txt
0       courses/cs252/projects/papers/buylow.txt
0       courses/cs252/projects/papers/.ipynb_checkpoints/buylow-checkpoint.txt
3       courses/cs252/projects/papers/.ipynb_checkpoints
8       courses/cs252/projects/papers
2       courses/cs252/projects

65

The `disk_usage` function is $O(n)$, for $n=$ number of files/folders
To prove this, it is e.t.s that 

* there are $n$ recursive calls of `disk_usage`
* the for loop is called $n-1$ times **across** all recursive calls of `disk_usage`

For clarity, consider the below hypothetical filesystem:

![](images/du.png)

* To prove that there are $n$ recursive calls of `disk_usage`, we need to define the _nesting level_ of each level. 
* (e.g. `/usr/rt/courses/` is level 0, `cs252/` is level 1, ...).
* We prove this by induction over the _nesting level_ $k$:
    * For $k=0$, the only folder at _nesting level_ $0$ is the initial path argument to `disk_usage` (`user/rt/courses` in the example). `disk_usage` is called once, as the main function call, the claim is satisfied.
    * If `disk_usage` is called for each file/folder at _nesting level_ $k-1$, we see that `disk_usage` will be called for each file/folder at _nesting level_ $k$ since each file/folder appears in the for loop.

Hence there are $n$ recursive calls of `disk_usage`, as needed.


We modify `disk_usage` to record how many files we loop over **across** recursive calls of `disk_usage`. 

We use this to verify that we loop over $n-1$ files in total

In [17]:
import os

def disk_usage(path):
    """Return the number of bytes used by a file/folder and any descendents."""
    global i
    total = os.path.getsize(path)                                                # account for direct usage
    if os.path.isdir(path):                                                      # if this is a directory
        visible_files = [f for f in os.listdir(path) if not f.startswith('.')]   # only include visible files
        i += len(visible_files)                                                  # record number of folders looped over
        print('looped over folders {}'.format(visible_files))
        for filename in visible_files:                                           # then for each child:
            childpath = os.path.join(path, filename)                             # compose full path to child
            total += disk_usage(childpath)                                       # add child’s usage to total
    #print ( '{0:<7}'.format(total), path)                                       # descriptive output (optional)
    return total                                                                 # return the grand total

In [18]:
i=0
disk_usage('courses/')
print('looped over {} folders'.format(i))

looped over folders ['cs016', 'cs252']
looped over folders ['homeworks', 'programs', 'grades']
looped over folders ['hw1.txt', 'hw3.txt', 'hw2.txt']
looped over folders ['pr3.txt', 'pr2.txt', 'pr1.txt']
looped over folders []
looped over folders ['grades', 'projects']
looped over folders []
looped over folders ['papers', 'demos']
looped over folders ['sellhigh.txt', 'buylow.txt']
looped over folders ['market.txt']
looped over 18 folders


Indeed, $19-1=18$

## 8. Inefficient Use of Recursion

Now for an example of an inefficient use of Recursion.
Notice how this is $O(2^n)$

In [19]:
def unique(S, start, stop):
    """Return True if there are no duplicate elements in slice S[start:stop]."""
    global num_steps
    num_steps += 1
    if stop - start <= 1: 
        return True # at most one item
    elif not unique(S, start, stop-1): 
        return False # first part has duplicate
    elif not unique(S, start+1, stop): 
        return False # second part has duplicate
    else: 
        return S[start] != S[stop-1] # do first and last differ?

In [20]:
num_steps = 0
unique('ABCDEFG',0,7)
print('took {} steps'.format(num_steps))

took 127 steps


In [21]:
2**len('ABCDEFG')-1

127

Now lets see why this is so slow

In [22]:
def unique(S, start, stop):
    """Return True if there are no duplicate elements in slice S[start:stop]."""
    global num_steps
    num_steps += 1
    print('analyzing sequence {}'.format(S[start:stop]))
    if stop - start <= 1: 
        print('one item left, so True')
        return True # at most one item
    elif not unique(S, start, stop-1): 
        print('sequence {} not unique'.format(S[start:stop-1]))
        return False # first part has duplicate
    elif not unique(S, start+1, stop): 
        print('sequence {} not unique'.format(S[start+1:stop]))
        return False # second part has duplicate
    else: 
        print('sequences {} and {} not unique. Checking if {} and {} are'.format(S[start:stop-1], S[start+1:stop], S[start], S[stop-1]))
        return S[start] != S[stop-1] # do first and last differ?
    
num_steps = 0
unique('ABC',0,3)
print('took {} steps'.format(num_steps))

analyzing sequence ABC
analyzing sequence AB
analyzing sequence A
one item left, so True
analyzing sequence B
one item left, so True
sequences A and B not unique. Checking if A and B are
analyzing sequence BC
analyzing sequence B
one item left, so True
analyzing sequence C
one item left, so True
sequences B and C not unique. Checking if B and C are
sequences AB and BC not unique. Checking if A and C are
took 7 steps


Notice that
* during `unique('ABC')`, we called **both** `unique('AB')` and `unique('BC')`.
* during `unique('AB')` called **both** `unique('A')` and `unique('B')`
* during `unique('BC')` called **both** `unique('B')` and `unique('C')`

Hence $1 + 2 + 2*2 = 7$ steps

## 9. Different Kinds of Recursion

![](images/diff_recursions_summary.png)

## Practice Problems

R 4-7. Reimplementing `int` using recursion

In [23]:
def str_to_int(string):
    """convert string to int e.g. '13531' -> 13531"""
    if len(string) == 1:
        return int(string)
    else:
        return int(string[0])*10**(len(string)-1) + str_to_int(string[1:])

In [24]:
str_to_int('13531')

13531

R 4.8 Isabel has an interesting way of summing up the values in a sequence A of
n integers, where n is a power of two.

She creates a new sequence B of half the size of A and sets $B[i] = A[2i] + A[2i+1]$, for $i = 0,1, ... , (n/2)-1$. 

If B has size 1, then she outputs $B[0]$. Otherwise, she replaces A with B, and repeats the process. 

What is the running time of her algorithm?

In [25]:
def isabel_sum(A):
    global idx
    B = [0]*(len(A)//2)
    for i in range(len(A)//2):
        idx += 1
        B[i] = A[2*i] + A[(2*i)+1]
    print(B)
    if len(B)==1:
        return B[0]
    else:
        return isabel_sum(B)

In [26]:
idx = 0
A = [1,2,4,5,7,3,0,8,1,2,4,5,7,3,0,8]

isabel_sum(A)

[3, 9, 10, 8, 3, 9, 10, 8]
[12, 18, 12, 18]
[30, 30]
[60]


60

We see there are $\log n$ recursive calls. However, altogether, these perform $n$ operations.
Hence this algorithm is $O(n)$

We can prove this more rigorously by noticing that the number of computations in total is:

$\sum_{k=1}^{\log n} \frac{n}{2^k} = \frac{n}{2} \sum_{k=0}^{(\log n) -1} \frac{1}{2^k} = \frac{n}{2}\left(\frac{(1-\frac{1}{n})}{(1-\frac{1}{2})}\right)=n-1$

Where we used the fact that this sum is a geometric series.

In [27]:
print('took {} steps (n={})'.format(idx, len(A)))

took 15 steps (n=16)
